In [280]:
import timeit
import re
from operator import add

startt = timeit.default_timer()

path = '/export/usuarios01/sblanco/Datos/repogit/spiff_spark/'
path_out = '/export/usuarios01/sblanco/Datos/repogit/spiff_spark/Output/salida.csv'

In [281]:
def countWords ( cad ):
    return ( [word for word in re.findall(r'\b[^\W\d_]+\b',cad)])


def getData ( cad ):
    
        autor = '-1'
        match_menciones = []
        match_etiquetas = []
        match_urls = []
        
        salida = []
        try:
            cad = cad.strip ()
            
            #El autor es la primera arroba:    
            match = re.search (r'@[\w\.-]+', cad) 
            autor =  match.group(0)
            
            
            #Dejamos la cadena sin el autor para seguir buscando en ella:
            inicio_autor = cad.find('@') + 1
            fin_autor = cad.find(' ',inicio_autor)
            cad = cad[fin_autor+1:]

            #Buscamos todas las menciones que aparezcan:
            match_menciones = re.findall(r'@[\w\.-]+', cad)
            
            #y las etiquetas:
            match_etiquetas = re.findall(r'#[\w\.-]+', cad)

            #y las urls:
            match_urls = re.findall(r'(https?://[^\s]+)',cad)
            
            
            
            
        except Exception as e:
                match_menciones = [str(e)]
                match_urls = [cad]
        
        
        salida = [ match_menciones, match_etiquetas, match_urls ]
            
    
        #salida[0]: autor
        #salida[1][0]: lista de menciones, salida[1][0][0] primera mencion
        #salida[1][1]: etiquetas
        return autor, salida
    
def toCSV (rd_tweets, f):
    
    datasep = ';'
    listsep = ','
    
    myF = open (f, 'w')
    
    for tw in rd_tweets.collect():
        line = tw[0] + datasep
        
        for menc in tw[1][0]:
            line = line + menc + listsep
            
        line = line + datasep
        
        for etiqu in tw[1][1]:
            line = line + etiqu + listsep
        
        myF.write (line + '\n')
        
    myF.close()

    

In [282]:
#tweets = sc.textFile('file:///export/usuarios01/sblanco/Datos/repogit/spiff_spark/Data/03_10_2016/red.txt')
file_path = 'file://'+ path + '/Data/03_10_2016/todo.txt'
tweets = sc.textFile(file_path)

alldata = tweets.map ( getData )

#Errores procesados, o buscar un autor concreto:
author = '-1'
print 'tweets fallidos: %s de %s' % (alldata.filter (lambda x : x[0]==author).count(), alldata.count())

tweetsValidos = alldata.filter (lambda x : x[0] <> '-1')

#Autor con mas entradas:
count = 10
moreentr = tweetsValidos.map ( lambda x: [x[0],1]).reduceByKey ( add ).takeOrdered( count ,key=lambda x: -x[1])
print 'Los %s autores con mas entradas son:%s' % (str(count), str(moreentr))

toCSV ( tweetsValidos, path_out)

tweets fallidos: 7879 de 98756
Los 10 autores con mas entradas son:[(u'@TunisiaTrends', 1815), (u'@SBZ_news', 1638), (u'@nawaat', 1524), (u'@Dima_Khatib', 1482), (u'@weddady', 1466), (u'@LiveWordCanada', 1234), (u'@halmustafa', 1149), (u'@TunObs', 1114), (u'@WikiActions', 958), (u'@PrimozVallant', 954)]


In [283]:
#expandimos la lista de etiquetas y las agrupamos
listetique = tweetsValidos.flatMap ( lambda x : x[1][1]).map (lambda x : (x,1)).reduceByKey(add)
topTenEtiq = listetique.takeOrdered( count ,key=lambda x: -x[1])

print 'Las %s etiquetas mas utilizadas son: %s' % (str(count), str(topTenEtiq))
print 'Etiquetas totales %s' % ( listetique.count() )

Las 10 etiquetas mas utilizadas son: [(u'#sidibouzid', 56296), (u'#SidiBouzid', 19533), (u'#Tunisia', 10806), (u'#Sidibouzid', 8323), (u'#tunisia', 5244), (u'#Tunisie', 3292), (u'#tunisie', 3131), (u'#OpTunisia', 2561), (u'#jasminrevolt', 2347), (u'#optunisia', 1523)]
Etiquetas totales 4346


In [284]:
#expandimos la lista de urls y las agrupamos
listUrl = tweetsValidos.flatMap ( lambda x : x[1][2]).map (lambda x : (x,1)).reduceByKey(add)
topTenUrl = listUrl.takeOrdered( count ,key=lambda x: -x[1])
print 'Las %s urls mas utilizadas son: %s' % (str(count), str(topTenUrl))
print 'Urls totales %s' % ( listUrl.count() )

Las 10 urls mas utilizadas son: [(u'http://wp.me/sEAit-blogger', 97), (u'http://bbc.in/ht3DvL', 90), (u'http://www.diigo.com/user/elhamalawy/Algeria', 80), (u'http://www.diigo.com/us...', 78), (u'http://tinyurl.com/5u4fzea', 53), (u'http://tinyurl.com/62s2gtd', 48), (u'http://curated.by/b/1S', 46), (u'http://tinyurl.com/6d9j88y', 46), (u'http://bit.ly/gouE0W', 43), (u'http://j.mp/gr6nTQ', 41)]
Urls totales 18328


In [285]:
print 'Total time:%s' % str ((timeit.default_timer() - startt)/60)

Total time:0.120576135317
